In [1]:
import mlflow
import dagshub

# Step 1: Initialize DagsHub with MLflow
repo_owner = "MohammoD2"  # Your DagsHub username
repo_name = "Youtube-Comment-Analysis-"  # Your DagsHub repository name
mlflow_tracking_uri = f"https://dagshub.com/{repo_owner}/{repo_name}.mlflow"

dagshub.init(repo_owner=repo_owner, repo_name=repo_name, mlflow=True)
mlflow.set_tracking_uri(mlflow_tracking_uri)

# Step 2: Set or create an experiment
experiment_name = "Exp 5 - ML Algos with HP Tuning"
mlflow.set_experiment(experiment_name)

Accessing as MohammoD2

Initialized MLflow to track repo "MohammoD2/Youtube-Comment-Analysis-"

Repository MohammoD2/Youtube-Comment-Analysis- initialized!

<Experiment: artifact_location='mlflow-artifacts:/c8ea385a5aba45869afef68984881de0', creation_time=1732855093258, experiment_id='5', last_update_time=1732855093258, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


d:\Work file\youtube_comment_analysis_v1\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [4]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Logistic Regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # LogisticRegression model setup with balanced class weight
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LogisticRegression"
    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Logistic Regression
run_optuna_experiment()


[I 2024-11-29 10:41:36,413] A new study created in memory with name: no-name-1f982b3b-5bcf-47fb-ab8d-77d7ec5eb793
[I 2024-11-29 10:41:36,758] Trial 0 finished with value: 0.6792432889452547 and parameters: {'C': 0.012688081324949245, 'penalty': 'l2'}. Best is trial 0 with value: 0.6792432889452547.
[I 2024-11-29 10:41:37,399] Trial 1 finished with value: 0.7694990488268865 and parameters: {'C': 2.8953510118755252, 'penalty': 'l2'}. Best is trial 1 with value: 0.7694990488268865.
[I 2024-11-29 10:41:37,687] Trial 2 finished with value: 0.7598816317903192 and parameters: {'C': 0.4923294352654759, 'penalty': 'l2'}. Best is trial 1 with value: 0.7694990488268865.
[I 2024-11-29 10:41:37,749] Trial 3 finished with value: 0.3333333333333333 and parameters: {'C': 0.00038275474647412716, 'penalty': 'l1'}. Best is trial 1 with value: 0.7694990488268865.
[I 2024-11-29 10:41:38,230] Trial 4 finished with value: 0.7722468822659058 and parameters: {'C': 4.242196620964991, 'penalty': 'l2'}. Best is t

🏃 View run LogisticRegression_SMOTE_TFIDF_Trigrams at: https://dagshub.com/MohammoD2/Youtube-Comment-Analysis-.mlflow/#/experiments/5/runs/058bb6c4d3d647b4b377706c177646c9
🧪 View experiment at: https://dagshub.com/MohammoD2/Youtube-Comment-Analysis-.mlflow/#/experiments/5
